In [1]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import r_regression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
class ClusteringTransformer:
    """
    This object does the clustering and weighting of FCA. Given features and a target, it:
    - computes the similarity matrix between features
    - uses this similarity matrix to cluster the features
    - trains a linear model to weigh the clusters

    It also follows the sklearn standard, so it can be used with sklearn utilities
    """

    def __init__(self, clusterer, metric=None, lr=True):
        """
        Create a ClusteringTransformer object
        :param clusterer: The clustering algorithm (must follow sklearn standard)
        :param metric: The similarity metric. Uses R^2 by default
        :param lr: Whether to use Logistic Regression or LDA
        """
        self.clusterer = clusterer
        if metric is None:
            self.metric = lambda X, y: r_regression(X, y) ** 2
        else:
            self.metric = metric
        self.use_lr = lr  # Use LR (as opposed to LDA)
        self.feature_loadings = None  # Holds the LR/LDA for each feature cluster
        self.feature_masks = None  # Holds assignments of features to clusters
        self.n_clusters = None  # Holds the number of clusters

    def fit(self, X, y):
        """
        Fit the model
        :param X: The features (n_samples, n_features)
        :param y: The target (n_samples,)
        :return: A fitted model
        """
        # Step 1 - construct a similarity matrix
        similarity_matrix = np.empty(shape=(X.shape[1], X.shape[1]))
        for i in range(X.shape[1]):
            similarity_matrix[:, i] = self.metric(X, X[:, i])

        # Step 2 - cluster based on the similarity matrix
        self.clusterer.fit(similarity_matrix)
        labels = np.unique(self.clusterer.labels_, return_inverse=True)[1]
        self.n_clusters = labels.max() + 1

        # Step 3 - extract loadings for our task
        self.feature_loadings = np.empty(shape=(self.n_clusters,), dtype=object)
        self.feature_masks = np.empty(shape=(self.n_clusters, X.shape[1]), dtype=bool)
        for i in range(self.n_clusters):
            mask = labels == i
            self.feature_masks[i, :] = mask

            # Run LDA/LR to dimension-reduce this feature set
            if self.use_lr:
                model = LogisticRegression()
                model.fit(X[:, mask], y)
            else:
                model = LinearDiscriminantAnalysis(n_components=1, shrinkage='auto', solver='eigen')
                model.fit(X[:, mask], y)
            self.feature_loadings[i] = model

        return self

    def transform(self, X):
        """
        Transform data using the fitted model
        :param X: The features to transform (n_samples, n_features)
        :return: The transformed features (n_features, n_clusters)
        """
        Xt = np.empty(shape=(X.shape[0], self.n_clusters))

        for i, (loading, mask) in enumerate(zip(self.feature_loadings, self.feature_masks)):
            if self.use_lr:
                Xt[:, i] = (self.feature_loadings[i].coef_[0][None, :] * X[:, mask]).sum(axis=1)
            else:
                Xt[:, i] = self.feature_loadings[i].transform(X[:, mask])[:, 0]
        return Xt

In [3]:
# Read in data
meta_df = pd.read_csv('bawe_corpus.csv', index_col=1)

# Grab target
labels = meta_df['grade'].values
target = 1 - np.unique(labels, return_inverse=True)[1]

# Grab features
docuscope_df = pd.read_csv('bawe_feats/docuscope_ngram/1.csv', index_col=0)
pos_df = pd.read_csv('bawe_feats/pos_ngram/1.csv', index_col=0)
# pos2_df = pd.read_csv('bawe_feats/pos_ngram/2.csv', index_col=0)
# docu2_df = pd.read_csv('bawe_feats/docuscope_ngram/2.csv', index_col=0)
biber_df = pd.read_csv('bawe_feats/biber.csv', index_col=0)


fdf = docuscope_df.join(biber_df, lsuffix='docu', rsuffix='biber')
fdf = fdf.join(pos_df, rsuffix='pos')
# fdf = fdf.join(pos2_df, rsuffix='pos 2')
# fdf = fdf.join(docu2_df, rsuffix='docu 2')

fdf = fdf.loc[meta_df.index]
feature_names = np.array(fdf.columns)


features = fdf.values
# features = features.astype(float)
features = (features - features.mean(axis=0)) / features.std(axis=0)
xmask = ~np.isnan(features.sum(axis=0))
features = features[:, xmask]
feature_names = feature_names[xmask]

In [4]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score

# Generate 10 random 10-fold splits. This is a bit overkill.
splits = []
for i in range(10):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=i)
    splits += list(skf.split(np.empty_like(target), target))

def evaluate_on_split(model):
    acs, bas, aucs = [], [], []
    for train, test in splits:
        model.fit(features[train], target[train])
        y_hat = model.predict(features[test])
        y_prob = model.predict_proba(features[test])[:, 1]

        acs.append(accuracy_score(target[test], y_hat))
        bas.append(balanced_accuracy_score(target[test], y_hat))
        aucs.append(roc_auc_score(target[test], y_prob))
    return acs, bas, aucs

In [6]:
# Our FCA Model
from sklearn.pipeline import Pipeline
from sklearn.cluster import AffinityPropagation

fca_model = Pipeline([
    ('ct', ClusteringTransformer(AffinityPropagation(affinity='precomputed', max_iter=1000), lr=True)),
    ('pred', LogisticRegression(class_weight='balanced'))
])

# PCA Model
from sklearn.decomposition import PCA

pca_model = GridSearchCV(
    estimator=Pipeline([
        ('pca', PCA()),
        ('pred', LogisticRegression(class_weight='balanced'))
    ]),
    param_grid={'pca__n_components': [2, 3, 5, 10, 15, 20,]}
)

# Factor Analysis Model
from sklearn.decomposition import FactorAnalysis

fa_model = GridSearchCV(
    estimator=Pipeline([
        ('fa', FactorAnalysis()),
        ('pred', LogisticRegression(class_weight='balanced'))
    ]),
    param_grid={'fa__n_components': [2, 3, 5, 10, 15, 20,]}
)

# No Dimensionality Reduction Model
simple_model = LogisticRegression(max_iter=1000, class_weight='balanced')

models = {
    'FCA': fca_model,
    'PCA': pca_model,
    'Factor Analysis': fa_model,
    'Base Features': simple_model,
}

In [7]:
model_info = []

# run the evaluations for each model. this takes some time!
for name, model in models.items():
    print(f'Running {name}...')
    acs, bas, aucs = evaluate_on_split(model)
    print(np.mean(acs), np.mean(bas), np.mean(aucs))
    model_info.append([name, np.mean(acs), np.std(acs), np.mean(bas), np.std(bas), np.mean(aucs), np.std(aucs)])

model_df = pd.DataFrame(
    model_info,
    columns=[
        'Method',
        'Avg. Acc.', 'Std. Acc.',
        'Avg. BA', 'Std. BA',
        'Avg. ROC AUC', 'Std. ROC AUC',
    ]
)
model_df

Running FCA...
0.6047081081081082 0.5895142156862745 0.6240868856209151
Running PCA...
0.5883261261261261 0.5805906862745097 0.6188971601307188
Running Factor Analysis...
0.5970414414414413 0.5873803921568627 0.6215499575163398
Running Base Features...
0.590854054054054 0.5766676470588236 0.5996369379084967


,Method,Avg. Acc.,Std. Acc.,Avg. BA,Std. BA,Avg. ROC AUC,Std. ROC AUC
0,FCA,0.604708,0.052521,0.589514,0.055244,0.624087,0.059546
1,PCA,0.588326,0.048206,0.580591,0.050372,0.618897,0.056659
2,Factor Analysis,0.597041,0.052741,0.587380,0.056139,0.621550,0.056956
3,Base Features,0.590854,0.051226,0.576668,0.054691,0.599637,0.065523


In [8]:
model_df.to_csv('performance.csv', index=False)

# Feature Loadings

Training our clustering on the entire dataset to get feature loadings.

In [11]:
def lr_CI(X, y, model, n=1000, alpha=0.95, p=0.90, rs=10):
    """
    Uses bootstrapping to estimate a p-value
    """
    np.random.seed(rs)
    m = deepcopy(model)
    vals = np.empty(shape=(X.shape[1], n))
    for i in range(n):
        # Take a subset of the model and get the coefficients again
        mask = np.random.choice(2, size=(X.shape[0],), p=[1 - p, p]).astype(bool)
        m.fit(X[mask], y[mask])
        vals[:, i] = m.coef_[0]

    # Calculate the lower and upper confidence bounds
    lower_idx, upper_idx = int(n * (1 - (alpha / 2))), int(n * (alpha / 2))
    vals.sort(axis=1)
    return ~((vals[:, lower_idx] <= 0.0) & (vals[:, upper_idx] >= 0.0))


In [12]:
# Re-train the model on the entire dataset
ct = ClusteringTransformer(AffinityPropagation(affinity='precomputed'), lr=True)
ct.fit(features, target)
transformed_features = ct.transform(features)

# Train the logisitc regression head
model = LogisticRegression(class_weight='balanced')
model.fit(transformed_features, target)
model_coefficients = model.coef_[0]
macro_significance = lr_CI(transformed_features, target, model)  # significance of each loading

# We write all the loadings to an Excel file, with each sheet representing a new loading
writer = pd.ExcelWriter("fca_loadings.xlsx", engine="xlsxwriter")
coef_dict = dict()

# Iterate over the loadings
for i, (coef, mask, model, sig) in enumerate(zip(model_coefficients, ct.feature_masks, ct.feature_loadings, macro_significance), start=1):
    local_coef = model.coef_[0]  # the coefficients for this specific loading
    significant_95 = lr_CI(features[:, mask], target, model, alpha=0.95, n=1000)  # run bootstrapping

    # put everything in a nice table
    mf_df = pd.DataFrame(
        np.array((feature_names[mask], local_coef, local_coef * coef, significant_95)).T,
        columns=['Feature', 'Macro-feature Coefficient', 'Overall Coefficient', 'Significant (95% CI)']
    )
    mf_df = mf_df.iloc[np.argsort(-local_coef)].reset_index(drop=True)  # sort values for easier viewing
    if coef != 0:
        # display significant loadings
        display(mf_df)

    # write to the Excel file. Insignificant loadings get marked with '(INSIG)' in their sheet names
    mf_df.to_excel(writer, sheet_name=f"{'(INSIG) ' if not sig else ''}Loading {i} | {round(coef, 5)}", index=False)
    coef_dict.update(dict(zip(feature_names[mask], local_coef * coef)))

writer.close()

,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,"('CitationHedged',)",0.100174,-0.016233,True
1,K57.suasiveVerbs,0.031965,-0.00518,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,"('ADP',)",0.497501,0.310336,True
1,"('PROPN',)",0.468559,0.292282,True
2,"('PUNCT',)",0.420398,0.26224,True
3,"('PART',)",0.27818,0.173526,True
4,"('SPACE',)",0.108548,0.067711,True
5,"('X',)pos",0.079206,0.049408,True
6,"('SYM',)",0.02095,0.013069,True
7,"('NUM',)",0.001617,0.001009,True
8,"('X',)",-0.004802,-0.002995,True
9,"('Interactive',)",-0.044485,-0.027749,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,B05.timeAdverbials,0.134163,0.092771,True
1,B04.placeAdverbials,0.000315,0.000218,True
2,H36.concessives,-0.022015,-0.015223,True
3,A03.presVerbs,-0.052568,-0.03635,True
4,A01.pastVerbs,-0.168765,-0.116697,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,"('FirstPerson',)",0.206661,0.076795,True
1,C06.1persProns,-0.131908,-0.049017,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,E16.Nouns,0.054592,-0.038385,True
1,N59.contractions,0.044821,-0.031514,True


...

,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,C10.demonstrProns,0.150898,0.079046,True
1,J43.TTR,0.140229,0.073457,True
2,P67.analNegn,0.114513,0.059986,True
3,I41.predAdj,0.00526,0.002755,True
4,K45.conjuncts,-0.086199,-0.045154,True
5,C09.impersProns,-0.138328,-0.072461,True
6,H35.causative,-0.184247,-0.096515,True
7,G19.beAsMain,-0.211844,-0.110972,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,N60.thatDeletion,0.439997,0.202943,True
1,H25.presPartClaus,0.259685,0.119776,True
2,K55.publicVerbs,-0.069969,-0.032272,True
3,K56.privateVerbs,-0.348816,-0.160887,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,N61.strandedPrep,0.015244,0.001237,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,"('ADV',)",0.49972,0.309917,True
1,"('Inquiry',)",0.351652,0.218088,True
2,"('ConfidenceHedged',)",0.165488,0.102633,True
3,"('PRON',)",0.115121,0.071396,True
4,"('Contingent',)",0.102237,0.063405,True
5,"('Uncertainty',)",0.09069,0.056244,True
6,P66.syntNegn,0.02481,0.015387,True
7,"('Updates',)",0.020028,0.012421,True
8,"('INTJ',)",-0.017986,-0.011155,True
9,"('Future',)",-0.021276,-0.013195,True


,Feature,Macro-feature Coefficient,Overall Coefficient,Significant (95% CI)
0,"('AcademicTerms',)",0.475837,0.176297,True
1,"('VERB',)",0.275175,0.101952,True
2,"('Citation',)",0.214097,0.079323,True
3,"('InformationExposition',)",0.19503,0.072259,True
4,"('MetadiscourseInteractive',)",0.141426,0.052398,True
5,"('AcademicWritingMoves',)",0.103416,0.038315,True
6,"('Facilitate',)",0.101913,0.037759,True
7,"('InformationChangeNegative',)",0.095973,0.035558,True
8,"('InformationChangePositive',)",0.06447,0.023886,True
9,"('CitationAuthority',)",0.01802,0.006676,True


# Coefficient Comparison
As FCA (our method), PCA, and Factor Analysis are all linear methods, we can get overall coefficients per each!
This is really just multiplying out the coefficients for multiple sequential linear transformations.

In [13]:
# some clustering methods don't put features in any cluster, so we set those features to zero weight
fca_coefficients = np.array([coef_dict[name] if name in coef_dict else 0.0 for name in feature_names])

In [25]:
# PCA
pca_model = GridSearchCV(
    estimator=Pipeline([
        ('pca', PCA()),
        ('pred', LogisticRegression(class_weight='balanced'))
    ]),
    param_grid={'pca__n_components': [2, 3, 5, 10, 15, 20,]}
)

pca_model.fit(features, target)


pca_coefficients = (pca_model.best_estimator_['pred'].coef_ @ pca_model.best_estimator_['pca'].components_)[0]

In [26]:
# FA
fa_model = GridSearchCV(
    estimator=Pipeline([
        ('fa', FactorAnalysis()),
        ('pred', LogisticRegression(class_weight='balanced'))
    ]),
    param_grid={'fa__n_components': [2, 3, 5, 10, 15, 20,]}
)
fa_model.fit(features, target)
fa_coefficients = (fa_model.best_estimator_['pred'].coef_ @ fa_model.best_estimator_['fa'].components_)[0]

In [27]:
coef_df = pd.DataFrame(
    np.array((feature_names, fca_coefficients, pca_coefficients, fa_coefficients,
              np.abs(fca_coefficients), np.abs(pca_coefficients), np.abs(fa_coefficients))).T,
    columns=['Feature', 'FCA Coef.', 'PCA Coef.', 'FA Coef.',
             'Abs FCA Coef.', 'Abs PCA Coef.', 'Abs FA Coef.']
)
coef_df

,Feature,FCA Coef.,PCA Coef.,FA Coef.,Abs FCA Coef.,Abs PCA Coef.,Abs FA Coef.
0,"('AcademicTerms',)",0.176297,0.039882,0.159494,0.176297,0.039882,0.159494
1,"('AcademicWritingMoves',)",0.038315,0.00994,0.038004,0.038315,0.00994,0.038004
2,"('Character',)",-0.235302,-0.005888,0.079054,0.235302,0.005888,0.079054
3,"('Citation',)",0.079323,0.019483,0.122169,0.079323,0.019483,0.122169
4,"('CitationAuthority',)",0.006676,-0.013826,0.068928,0.006676,0.013826,0.068928
...,...,...,...,...,...,...,...
98,"('SCONJ',)",-0.076354,-0.02366,0.034065,0.076354,0.02366,0.034065
99,"('SPACE',)",0.067711,0.045629,0.126262,0.067711,0.045629,0.126262
100,"('SYM',)",0.013069,0.027158,0.172491,0.013069,0.027158,0.172491
101,"('VERB',)",0.101952,0.003886,0.097191,0.101952,0.003886,0.097191


In [28]:
coef_df.to_csv('coefficients.csv', index=False)